# Web Scraping

En este proyecto trabajaremos la extracción automática de datos web, proceso conocido como Web Scraping.

Las principales metodologías que nos permiten realizar Web Scraping son:

- Utilizando API's (Application programming interface)
- Mediante Scrapers (librerías que nos permiten extraer datos web automáticamente. Por ejemplo, Beautiful-Soup, Requests-html,...)
- Web Crawling (un programa 'spider' busca por el contenido solicitado siguiendo links y explorándolos).

En nuestro caso, vamos a utilizar API's y Scrapers para realizar el proyecto.

## Web Scraping con API's

Una API es una interfaz de programación de aplicaciones. Esta especifica cómo deben interactuar cliente y servidor. Si el cliente realiza una solicitud en un formato específico, el servidor siempre responderá en un formato documentado o iniciará una acción.

Algunas API son gratuitas, pero la mayoría suelen ser de pago o necesitan registro. Normalmente, se proporciona una clave (KEY) y una identificación (IDENTIFICATION_KEY) que debe incorporarse en cada solicitud a esa API. Por último, algunas API de pago ofrecen también una versión gratuita disponible para fines educativos o académicos, prohibiendo la comercialización de productos que hagan su uso sin previo pago de licencia.

Para realizar las consultas se requiere la previa revisión de la documentación que proporciona la API para conocer la sintaxis que requieren las consultas o solicitudes y cuáles son los códigos de estado relevantes de la misma.

Para este proyecto vamos a trabajar con una de las API's gratuitas que ofrece la NASA para obtener patentes relativas a distintos campos de la ciencia.

Para utilizar esta API, es necesario registrarse para obtener una clave (API_KEY) que necesitará ser incluida para realizar las diferentes solicitudes en la API.

Para registrarse, es necesario rellenar el formulario que aparece en la parte 'Generate API Key' de la web: https://api.nasa.gov/.

### Imagen del día de la NASA

La NASA en la página web [Astronomy Picture of the Day](https://www.google.com/url?q=https%3A%2F%2Fapod.nasa.gov%2Fapod%2Fastropix.html) ofrece una imagen diferente todos los días del año. 

A continuación, vamos a obtener los datos relativos a las imágenes que fueron "Imagen del día" para diferentes consultas.

Primero de todo, cargamos las librerías que vamos a usar, definimos la URL base y la API KEY.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import date
from IPython.core.display import HTML

In [2]:
url_base = "https://api.nasa.gov/planetary/apod"

In [ ]:
API_KEY = {
    "api_key": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
}

Ahora que tenemos al URL base y la API KEY definidas, como primera consulta vamos a obtener un dataframe de una fila que contenga la información de la imagen del día en que se está realizando este proyecto.

Empezamos definiendio una función para mostrar las imágenes dentro de un dataframe en protocolo HTML.

In [4]:
from IPython.core.display import HTML

def path_to_image_html(path):
    return '<img src="'+ path + '" width="100" >'

Seguidamente, hacemos la solicitud a la API d'APOD. Si la solicitud a la API se efectua correctamente, se almacenara la respuesta de formato JSON en la variable "data".

In [5]:
response = requests.get(url_base, params=API_KEY)

if response.status_code == 200:
  data = response.json()

Finalmente, mostramos el DataFrame con las imágenes en protocolo HTML.

In [6]:
df = pd.DataFrame([data])

def path_to_image_html(path):
  return '<img src="'+ path + '" width="100" >'

HTML(df.to_html(escape=False, formatters=dict(url=path_to_image_html)))

,copyright,date,explanation,hdurl,media_type,service_version,title,url
0,\nMartin LaMontagne\n,2025-02-17,"What's happened to the sky? Last Monday, the photogenic launch plume from a SpaceX rocket launch created quite a spectacle over parts of southern California and Arizona. Looking at times like a giant space fish, the impressive rocket launch from Vandenberg Air Force Base near Lompoc, California, was so bright because it was backlit by the setting Sun. The Falcon 9 rocket successfully delivered to low Earth orbit 23 Starlink communications satellites. The plume from the first stage is seen on the right, while the soaring upper stage rocket is seen at the apex of the plume toward the left. Venus appears at the top of the frame, while a bright streetlight shines on the far right. The featured image was captured toward the west after sunset from near Phoenix, Arizona.",https://apod.nasa.gov/apod/image/2502/FishPlume_LaMontagne_2272.jpg,image,v1,SpaceX Rocket Launch Plume over California,


El objetivo de la siguiente consulta va a ser obtener un DataFrame de una fila que contenga la información de la imagen del día de mi cumpleaños (en este caso, lo voy a hacer del año pasado, puesto que en el momento de realizar este proyecto, aún no existe tal imagen para tal fecha, ya que aún faltan unos meses para mi cumpleaños).

Empezamos definiendo la fecha del aniversario y la API KEY.

In [ ]:
birthday = date(2024, 3, 8)

API_KEY = {
    "api_key": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
    "date": birthday.strftime("%Y-%m-%d")
}

Realizamos la solicitud.

In [8]:
response = requests.get(url_base, params=API_KEY)

if response.status_code == 200:
  data = response.json()

Finalmente, mostramos el DataFrame con las imágenes en protocolo HTML.

In [9]:
df = pd.DataFrame([data])

def path_to_image_html(path):
  return '<img src="'+ path + '" width="100" >'

HTML(df.to_html(escape=False, formatters=dict(url=path_to_image_html)))

,copyright,date,explanation,hdurl,media_type,service_version,title,url
0,Robert Gendler,2024-03-08,"The Tarantula Nebula, also known as 30 Doradus, is more than a thousand light-years in diameter, a giant star forming region within nearby satellite galaxy the Large Magellanic Cloud. About 180 thousand light-years away, it's the largest, most violent star forming region known in the whole Local Group of galaxies. The cosmic arachnid sprawls across this magnificent view, an assembly of image data from large space- and ground-based telescopes. Within the Tarantula (NGC 2070), intense radiation, stellar winds, and supernova shocks from the central young cluster of massive stars cataloged as R136 energize the nebular glow and shape the spidery filaments. Around the Tarantula are other star forming regions with young star clusters, filaments, and blown-out bubble-shaped clouds. In fact, the frame includes the site of the closest supernova in modern times, SN 1987A, at lower right. The rich field of view spans about 2 degrees or 4 full moons in the southern constellation Dorado. But were the Tarantula Nebula closer, say 1,500 light-years distant like the Milky Way's own star forming Orion Nebula, it would take up half the sky.",https://apod.nasa.gov/apod/image/2403/Tarantula-HST-ESO-Webb-LL.jpg,image,v1,The Tarantula Zone,


Para la última consulta, vamos a obtener un DataFrame que contenga "las imágenes del día" de todos los días comprendidos entre un intervalo de tiempo. Una vez implementado el código, comprobaremos y mostraremos los resultados para el período comprendido entre 10 de enero de 2025 y 20 de enero de 2025.

Empezamos definiendo la URL BASE y la API KEY.

In [ ]:
url_base = "https://api.nasa.gov/planetary/apod"

API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [11]:
#start: 2025-01-10
#end: 2025-01-20

# Pedimos introducir una fecha inicial y final, estas fechas se guardarán respectivamente dentro de las variables "start" y "end".
start = input("Introduce la fecha de inicio (YYYY-MM-DD): ")
end = input("Introduce la fecha final (YYYY-MM-DD): ")

# Convertimos las fechas en objetos datetime.
start_date = pd.to_datetime(start)
end_date = pd.to_datetime(end)

# Convertimos las fechas a strings y realizamos las solicitudes a la API para cada fecha en el intervalo.
dataframes = []

while start_date <= end_date:
    data_str = start_date.strftime('%Y-%m-%d')
    params = {
        'date': data_str,
        'api_key': API_KEY
    }
    req = requests.get(url_base, params=params)
    data = req.json()
    df = pd.DataFrame([data])
    dataframes.append(df)
    start_date += pd.DateOffset(days=1)

result = pd.concat(dataframes)

# Seleccionamos las columnas.
result = result[['date', 'explanation', 'title', 'url']]

# Aplicamos la función path_to_image_html() a la columna url para generar código HTML que muestre las imágenes en una tabla.
result ['url'] = result ['url'].apply(path_to_image_html)

# Finalmente, mostramos el DataFrame con las imágenes en HTML.
HTML(result.to_html(escape=False, formatters={'url': path_to_image_html}))

,date,explanation,title,url
0,2025-01-10,"An unassuming region in the constellation Taurus holds these dark and dusty nebulae. Scattered through the scene, stars in multiple star systems are forming within their natal Taurus molecular cloud complex some 450 light-years away. Millions of years young and still going through stellar adolescence, the stars are variable in brightness and in the late phases of their gravitational collapse. Known as T-Tauri class stars they tend to be faint and take on a yellowish hue in the image. One of the brightest T-Tauri stars in Taurus, V773 (aka HD283447) is near the center of the telescopic frame that spans over 1 degree. Toward the top is the dense, dark marking on the sky cataloged as Barnard 209.","Young Stars, Dark Nebulae",""" width=""100"" >"
0,2025-01-11,"Only Mercury is missing from a Solar System parade of planets in this early evening skyscape. Rising nearly opposite the Sun, bright Mars is at the far left. The other naked-eye planets Jupiter, Saturn, and Venus, can also be spotted, with the positions of too-faint Uranus and Neptune marked near the arcing trace of the ecliptic plane. On the far right and close to the western horizon after sunset is a young crescent Moon whose surface is partly illuminated by earthshine. In the foreground of the composite panorama captured on 2 January, planet Earth is represented by Mount Etna's lower Silvestri Crater. Of course Earth's early evening skies are full of planets for the entire month of January. On 13 January, a nearly Full Moon will appear to pass in front of Mars for skywatchers in the continental U.S. and Eastern Canada.",An Evening Sky Full of Planets,""" width=""100"" >"
0,2025-01-12,"Whatever hit Mimas nearly destroyed it. What remains is one of the largest impact craters on one of Saturn's smallest round moons. Analysis indicates that a slightly larger impact would have destroyed Mimas entirely. The huge crater, named Herschel after the 1789 discoverer of Mimas, Sir William Herschel, spans about 130 kilometers and is featured here. Mimas' low mass produces a surface gravity just strong enough to create a spherical body but weak enough to allow such relatively large surface features. Mimas is made of mostly water ice with a smattering of rock - so it is accurately described as a big dirty snowball. The featured image was taken during the closest-ever flyby of the robot spacecraft Cassini past Mimas in 2010 while in orbit around Saturn. Interactive: Take a trek across Mimas January 14: Zoom APOD Lecture hosted by the Amateur Astronomers of Association of New York",Mimas: Small Moon with a Big Crater,""" width=""100"" >"
0,2025-01-13,"Comet ATLAS is really bright now, but also really close to the Sun. Outside the glow of the Sun, Comet C/2024 G3 (ATLAS) would be one of the more remarkable comet sights of recent years, reflecting about as much sunlight to Earth as Comet Tsuchinshan-ATLAS did in October, and now rivaling even planet Venus. But the giant snowball is now so close to the Sun that it can only be seen through the light of the early morning dawn or the early evening dusk. Today, Comet ATLAS is at perihelion -- its closest ever to the Sun. Although the future brightness of comets is notoriously hard to predict, there is hope that Comet ATLAS will survive its close pass near the Sun and remain bright enough to be seen with the unaided eye over the next few days -- and possibly a good camera comet for weeks. The featured image was taken early yesterday morning near Tornaľa, Slovakia. Tomorrow: Zoom APOD Lecture hosted by the Amateur Astronomers of Association of New York",Comet ATLAS Before Sunrise,""" width=""100"" >"
0,2025-01-14,"Why is Polaris called the North Star? First, Polaris is the nearest bright star toward the north spin axis of the Earth. Therefore, as the Earth turns, stars appear to revolve around Polaris, but Polaris itself always stays in the same northerly direction -- making it the North Star. Since no brig

## Web Scraping con Beautiful Soup

Beautiful Soup es una librería de Python para extraer datos de un documento HTML. Ésta se basa en analizar el HTML con la ayuda de un parser.

Un parser, en el contexto de la web scraping, es un programa diseñado para recorrer la estructura de árbol o de componentes anidados de un documento HTML de forma que se pueda identificar y acceder fácilmente a los distintos elementos de la misma.

Hay diferentes parsers disponibles que pueden ser utilizados y seleccionados para ser utilizado con Beautiful Soup.

- 'html.parser'
- 'lxml' (Es el mejor valorado por BeautifulSoap en la mayoría de los casos)
- 'html5lib'


En caso de no ser especificado ningún parser, Beautiful Soup elige el que considera más adecuado.

Una vez creado el objeto BeautifulSoap, es recomendable abrir el html generado para comprobar que se ha parseado correctamente. Es decir, que se ha recogido correctamente la estructura del documento HTML.

Los principales pasos que debemos seguir en un proyecto de web scraping con BeautifulSoup son los siguientes:

1. Cargar las librerías que se van a utilizar.

2. Inspeccionar la página que se desee scrapear (*).

3. Escoger el parser.

4. Crear el objeto BeaurifulSoup.

5. Exportar el HTML a un archivo para comprobar que el objeto BeautifulSoup se ha creado correctamente y/o dónde está la información de interés.

6. Determinar cuáles son las etiquetas del documento HTML que contiene la información que nos interesa scrapear.

7. Aplicar los métodos disponibles en la librería BeautifulSoup para obtener el contenido deseado.

8. Organizar el contenido scrapeado en la estructura deseada o más conveniente de acuerdo a los datos obtenidos.

Para inspeccionar una página web se recomienda utilizar el inspector de páginas web disponible en los navegadores. Una vez abierto el inspector, abrir la parte "Developer". Desde aquí se puede navegar por el documento HTML y buscar e identificar los elementos y etiquetas donde se encuentra el contenido que se desea scapear.

### Wikipedia

Vamos a ver cómo scrapear contenido de Wikipedia, concretamente, vamos a obtener la lista de las festividades más relevantes de España, los enlaces donde podemos encontrar más información y una breve descripción de las mismas.

Primero de todo, cargamos las librerías que vamos a usar y definimos la URL base.

In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

In [13]:
url_base = "https://es.wikipedia.org/wiki/Turismo_en_Espa%C3%B1a"

response = requests.get(url_base)
response.status_code

200

A continuación, extraemos el contenido del HTML y creamos un objeto BeautifulSoup que nos permita analizar y manipular el HTML usando el parser "html.parser".


In [14]:
html = response.content

soup = BeautifulSoup(html, "html.parser")

with open('Wiki.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

En el explorador de archivos del Collaboratory (desplegar el menú de la izquierda), podemos abrir el archivo generado Wiki_respuesta.html para revisar la estructura de la página web. Esto, junto con el inspector de páginas web disponible en el navegador, nos permitirá localizar dónde se encuentra la información que nos interesa (Festivos de España).

Si nos fijamos, podemos ver que la información de interés está contenida en los enlaces (\<a>) del último párrafo (\<p>).

Vamos a obtener todos los links del último párrafo.

In [15]:
links=soup.find_all('p')[-1].find_all('a')
links

[<a class="mw-redirect" href="/wiki/Fiestas_de_Inter%C3%A9s_Tur%C3%ADstico_Internacional_(Espa%C3%B1a)" title="Fiestas de Interés Turístico Internacional (España)">Fiestas de Interés Turístico Internacional</a>,
 <a href="/wiki/Feria_de_Abril" title="Feria de Abril">Feria de Abril de Sevilla</a>,
 <a href="/wiki/Hogueras_de_Alicante" title="Hogueras de Alicante">Hogueras de Alicante</a>,
 <a href="/wiki/Feria_de_Albacete" title="Feria de Albacete">Feria de Albacete</a>,
 <a href="/wiki/Carnaval_de_Santa_Cruz_de_Tenerife" title="Carnaval de Santa Cruz de Tenerife">Carnaval de Santa Cruz de Tenerife</a>,
 <a href="/wiki/Sanfermines" title="Sanfermines">Sanfermines de Pamplona</a>,
 <a href="/wiki/Fiestas_del_Pilar" title="Fiestas del Pilar">Fiestas del Pilar de Zaragoza</a>,
 <a class="mw-redirect" href="/wiki/Semana_Santa_de_Sevilla" title="Semana Santa de Sevilla">Semana Santa de Sevilla</a>]

La URL que aparece asociada a cada festividad es una URL relativa. Por tanto, para obtener la dirección URL completa podemos utilizar la función "urljoin".

In [16]:
# Extraer el nombre de la festividad (asociado al atributo 'title' del elemento 'a').
links_names=[l.get('title') for l in links]

# Extraer el valor de la url relativa (asociada con el atributo 'href' del elemento 'a').
url_rel_links=[l.get('href') for l in links]

# Transformar los links relativos a absolutos.
url_complete_links = [urljoin(url_base, url) for url in url_rel_links]

Extraemos solo las URL que apuntan a Wikipedia (URL internas).

In [17]:
links_int = [url for url in url_complete_links if 'wikipedia.org' in url]
links_int

['https://es.wikipedia.org/wiki/Fiestas_de_Inter%C3%A9s_Tur%C3%ADstico_Internacional_(Espa%C3%B1a)',
 'https://es.wikipedia.org/wiki/Feria_de_Abril',
 'https://es.wikipedia.org/wiki/Hogueras_de_Alicante',
 'https://es.wikipedia.org/wiki/Feria_de_Albacete',
 'https://es.wikipedia.org/wiki/Carnaval_de_Santa_Cruz_de_Tenerife',
 'https://es.wikipedia.org/wiki/Sanfermines',
 'https://es.wikipedia.org/wiki/Fiestas_del_Pilar',
 'https://es.wikipedia.org/wiki/Semana_Santa_de_Sevilla']

Visualizamos los nombres de los links.

In [18]:
links_names

['Fiestas de Interés Turístico Internacional (España)',
 'Feria de Abril',
 'Hogueras de Alicante',
 'Feria de Albacete',
 'Carnaval de Santa Cruz de Tenerife',
 'Sanfermines',
 'Fiestas del Pilar',
 'Semana Santa de Sevilla']

Para poder obtener la breve descripción de cada una de las festividades asociadas a los links anteriores, vamos a scrapear el contenido de estas URL's y obtener la información contenida en el primer parágrafo (asumimos que en él es donde se hace la introducción del concepto).

Para realizarlo, construimos un loop que, en cada iteración, obtenga los datos de cada URL. Antes de cada solicitud en el servidro esperamos un segundo para controlar el tiempo y asegurarnos de que se pueden ejecutar correctamente las solicitudes.

In [19]:
# Inicializar la lista donde se guardarán los párrafos con la breve descripción.
par_text = []

# Bucle para scrapear cada link.
i = 0
for url in links_int:

    # Esperar 1 segundo antes de cada solicitud.
    time.sleep(1)

    # Conectarse a todas las páginas web.
    p_resp = requests.get(url)

    # Comprobar si la solicitud se ha realizado correctamente.
    if p_resp.status_code == 200:            # OK!
        print('URL #{0}: {1}'.format(i+1,url))    # Imprimir el número de iteración junto con la URL.

    else:
        print('Status code {0}: Skipping URL #{1}: {2}'.format(p_resp.status_code, i+1, url)) #Imprimir el error.
        i = i+1
        continue


    # Obtener HTML.
    p_html = p_resp.content

    # Convertir HTML a objeto BeautifulSoup.
    p_soup = BeautifulSoup(p_html, 'html.parser')

    # Obtener texto relativo al primer párrafo.
    p_pars = p_soup.find("p").text

    # Añadir el párrafo a la lista de párrafos.
    par_text.append(p_pars)

    # Incrementar el contador del bucle.
    i = i+1

URL #1: https://es.wikipedia.org/wiki/Fiestas_de_Inter%C3%A9s_Tur%C3%ADstico_Internacional_(Espa%C3%B1a)
URL #2: https://es.wikipedia.org/wiki/Feria_de_Abril
URL #3: https://es.wikipedia.org/wiki/Hogueras_de_Alicante
URL #4: https://es.wikipedia.org/wiki/Feria_de_Albacete
URL #5: https://es.wikipedia.org/wiki/Carnaval_de_Santa_Cruz_de_Tenerife
URL #6: https://es.wikipedia.org/wiki/Sanfermines
URL #7: https://es.wikipedia.org/wiki/Fiestas_del_Pilar
URL #8: https://es.wikipedia.org/wiki/Semana_Santa_de_Sevilla


Finalmente, creamos un diccionario donde cada URL en "links_int" se asocie con el párrafo correspondiente en "par_text" y mostramos el diccionario.

In [20]:
url_and_info = dict(zip(links_int, par_text))
url_and_info

{'https://es.wikipedia.org/wiki/Fiestas_de_Inter%C3%A9s_Tur%C3%ADstico_Internacional_(Espa%C3%B1a)': 'La declaración de Fiesta de Interés Turístico Internacional es una distinción de carácter honorífico que se concede en España por la Secretaría General de Turismo del Ministerio de Industria, Turismo y Comercio a las fiestas o acontecimientos que supongan manifestaciones de valores culturales y de tradición popular, con especial consideración a sus características etnológicas y que tengan una especial importancia como atractivo turístico.\n',
 'https://es.wikipedia.org/wiki/Feria_de_Abril': 'La Feria de Abril o Feria de Sevilla es una fiesta de primavera que se celebra anualmente en la ciudad de Sevilla (Comunidad autónoma de Andalucía, España), donde el público se reúne en un gran recinto denominado Real de la Feria, con calles engalanadas con farolillos y con casetas efímeras, por las que durante el día circulan jinetes y coches de caballo y por las que pasan diariamente aproximadame

### Rotten Tomatoes & IMDb

Tener una suscripción a una plataforma y no saber qué ver es un problema común. Existen diferentes páginas web que ofrecen información muy detallada y continuamente actualizada por usuarios sobre contenido audiovisual que ayuda a elegir cuál será la siguiente película a ver.

Entre toda la multitud de información que se puede consultar, para este proyecto nos centraremos en conocer las 100 mejores películas de la historia, así como las 100 mejores películas entre 2022 y 2024 según Rotten Tomatoes.

Las características que queremos para el DataFrame de Rotten Tomatoes son:
- Título
- Año
- Puntuación
- Director
- Crítica
- URL completa de la película (url_base + url_relativa)

La información scrapeada será organizada en un DataFrame de forma que cada característica esté dispuesta en una columna.

Cargamos las librerías que vamos a utilizar.

In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

El primer objetivo es obtener un DataFrame de las 100 mejores películas de la historia según Rotten Tomatoes.

In [22]:
# URL de la lista de las mejores películas de todos los tiempos.
url = "https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/"

response = requests.get(url)
response.status_code

200

In [23]:
# Analizar el contenido HTML.
soup = BeautifulSoup(response.content, "html.parser")

# Encontrar todas las entradas de películas en la lista.
movie_entries = soup.find_all("div", class_="col-sm-18 col-full-xs countdown-item-content")

# Lista para almacenar los datos de las películas.
movies_data = []

# Iterar sobre las primeras 100 películas.
for entry in movie_entries[:100]:
    # Extraer el título
    title_tag = entry.find('a')
    title = title_tag.get_text(strip=True) if title_tag else "Desconocido"

    # Extraer el año.
    year_tag = entry.find('span', class_='subtle start-year')
    year = year_tag.get_text(strip=True).strip('()') if year_tag else "Desconocido"

    # Extraer la puntuación.
    score_tag = entry.find('span', class_='tMeterScore')
    score_text = score_tag.get_text(strip=True).replace('%', '') if score_tag else ''
    score = int(score_text) if score_text.isdigit() else None  # Evita errores con valores vacíos.

    # Extraer el director.
    director_tag = entry.find('div', class_='info director')
    director = director_tag.get_text(strip=True).replace('Directed By:', '').strip() if director_tag else "Desconocido"

    # Extraer la crítica.
    critic_tag = entry.find('div', class_='info critics-consensus')
    critic = critic_tag.get_text(strip=True).replace('Critics Consensus:', '').strip() if critic_tag else "Sin crítica"

    # Construir la URL completa de la película.
    url_tag = title_tag['href'] if title_tag and title_tag.has_attr('href') else None
    full_url = url_tag if url_tag else "No disponible"

    # Agregar los datos de la película a la lista.
    movies_data.append({
        'Título': title,
        'Año': year,
        'Puntuación': score,
        'Director': director,
        'Crítica': critic,
        'URL': full_url
    })
  
# Crear un DataFrame de pandas con los datos recopilados.
df_movies = pd.DataFrame(movies_data)

# Eliminar filas con puntuaciones nulas antes de ordenar.
df_movies = df_movies.dropna(subset=["Puntuación"])

# Ordenar por puntuación de mayor a menor.
df_movies = df_movies.sort_values(by="Puntuación", ascending=False).reset_index(drop=True)

# Imprimir la tabla con las 100 mejores películas.
df_movies.head(100)

,Título,Año,Puntuación,Director,Crítica,URL
0,Toy Story 2,1999,100,"Ash Brannon,John Lasseter,Lee Unkrich",The rare sequel that arguably improves on its ...,https://www.rottentomatoes.com/m/toy_story_2
1,Seven Samurai,1954,100,Akira Kurosawa,"Arguably Akira Kurosawa's masterpiece, The Sev...",https://www.rottentomatoes.com/m/seven_samurai...
2,Toy Story,1995,100,John Lasseter,"Entertaining as it is innovative,Toy Storyrein...",https://www.rottentomatoes.com/m/toy_story
3,Cool Hand Luke,1967,100,Stuart Rosenberg,Though hampered by Stuart Rosenberg's directio...,https://www.rottentomatoes.com/m/cool_hand_luke
4,Before Sunrise,1995,100,Richard Linklater,"Thought-provoking and beautifully filmed,Befor...",https://www.rottentomatoes.com/m/before_sunrise
5,M,1931,100,Fritz Lang,A landmark psychological thriller with arresti...,https://www.rottentomatoes.com/m/1012928-m
6,Three Colors: Red,1994,100,Krzysztof Kieslowski,"A complex, stirring, and beautifully realized ...",https://www.rottentomatoes.com/m/three_colors_red
7,12 Angry Men,1957,100,Sidney Lumet,Sidney Lumet's feature debut is a superbly wri...,https://www.rottentomatoes.com/m/1000013_12_an...
8,Singin' in the Rain,1952,100,"Stanley Donen,Gene Kelly","Clever, incisive, and funny, Singin' in the Ra...",https://www.rottentomatoes.com/m/singin_in_the...
9,The Philadelphia Story,1940,100,George Cukor,"Offering a wonderfully witty script, spotless ...",https://www.rottentomatoes.com/m/philadelphia_...


El segundo objetivo es obtener un DataFrame de las 100 mejores películas entre 2022 y 2024 según Rotten Tomatoes.

In [24]:
# URLs de los años 2022, 2023 y 2024.
urls = {
    "2022": "https://editorial.rottentomatoes.com/guide/best-movies-2022/",
    "2023": "https://editorial.rottentomatoes.com/guide/best-movies-of-2023/",
    "2024": "https://editorial.rottentomatoes.com/guide/best-2024-movies-every-certified-fresh/"
}

In [25]:
# Lista para almacenar los datos de todas las películas.
movies_data = []

# Función para extraer datos de películas de una URL específica.
def extract_movies(url, year):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al acceder a {year}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    movie_entries = soup.find_all("div", class_="col-sm-18 col-full-xs countdown-item-content")
    
    extracted_movies = []
    for entry in movie_entries:
        # Extraer el título.
        title_tag = entry.find('a')
        title = title_tag.get_text(strip=True) if title_tag else "Desconocido"

        # Extraer la puntuación.
        score_tag = entry.find('span', class_='tMeterScore')
        score_text = score_tag.get_text(strip=True).replace('%', '') if score_tag else ''
        score = int(score_text) if score_text.isdigit() else None

        # Extraer el director.
        director_tag = entry.find('div', class_='info director')
        director = director_tag.get_text(strip=True).replace('Directed By:', '').strip() if director_tag else "Desconocido"

        # Extraer la crítica.
        critic_tag = entry.find('div', class_='info critics-consensus')
        critic = critic_tag.get_text(strip=True).replace('Critics Consensus:', '').strip() if critic_tag else "Sin crítica"

        # Construir la URL completa de la película.
        url_tag = title_tag['href'] if title_tag and title_tag.has_attr('href') else None
        full_url = url_tag if url_tag else "No disponible"

        # Agregar la película a la lista.
        extracted_movies.append({
            'Título': title,
            'Año': year,
            'Puntuación': score,
            'Director': director,
            'Crítica': critic,
            'URL': full_url
        })
    return extracted_movies

# Extraer películas de cada año y combinarlas.
for year, url in urls.items():
    movies_data.extend(extract_movies(url, year))

# Crear DataFrame con los datos obtenidos.
df_movies = pd.DataFrame(movies_data)

# Eliminar filas con puntuaciones nulas antes de ordenar.
df_movies = df_movies.dropna(subset=["Puntuación"])

# Ordenar por puntuación de mayor a menor.
df_movies = df_movies.sort_values(by="Puntuación", ascending=False).reset_index(drop=True)

# Imprimir la tabla con las 100 mejores películas.
df_movies.head(100)

,Título,Año,Puntuación,Director,Crítica,URL
0,Wallace & Gromit: Vengeance Most Fowl,2024,100,"Merlin Crossingham,Nick Park","Comforting as cheese and crackers, with some g...",https://www.rottentomatoes.com/m/wallace_and_g...
1,Girls Will Be Girls,2024,100,Shuchi Talati,A deeply felt coming of age tale that gleans i...,https://www.rottentomatoes.com/m/girls_will_be...
2,On Becoming a Guinea Fowl,2024,100,Rungano Nyoni,A vibrant exploration of family and social mor...,https://www.rottentomatoes.com/m/on_becoming_a...
3,Santosh,2024,100,Sandhya Suri,Shahana Goswami bristles with unforgettable in...,https://www.rottentomatoes.com/m/santosh_2024
4,Nowhere Special,2024,100,Uberto Pasolini,Focusing pragmatically on the ordinary human m...,https://www.rottentomatoes.com/m/nowhere_special
...,...,...,...,...,...,...
95,Blue Jean,2023,96,Georgia Oakley,Bridging times past with issues that are still...,https://www.rottentomatoes.com/m/blue_jean
96,"Lingui, The Sacred Bonds",2022,96,Mahamat-Saleh Haroun,"Lingui, the Sacred Bondsuses one family's frau...",https://www.rottentomatoes.com/m/lingui_the_sa...
97,Perfect Days,2023,96,Wim Wenders,An absorbing slice-of-life drama led by a rema...,https://www.rottentomatoes.com/m/perfect_days_...
98,The Long Walk,2022,96,Mattie Do,The Long Walkmay require patience from some vi...,https://www.rottentomatoes.com/m/the_long_walk
